In [1]:
import pandas as pd
import numpy as np
import random as rn


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold


from keras.preprocessing import sequence, text
from keras.models import Sequential,Model, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
#from keras import initializations
from keras import initializers, regularizers, constraints, optimizers
import tensorflow as tf
import keras

tf.set_random_seed(234)
np.random.seed(234)
rn.seed(234)

/home/olamide/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("SampleSubmission.csv")
df = train.append(test, ignore_index=True)

In [3]:
train.tail(5)

,Viewers_ID,Joke_identifier,Response_ID,Rating
612697,A9999,Gordons Lagos 2,A9999_Gordons Lagos 2,-0.92
612698,A9999,Gordons Lagos 3,A9999_Gordons Lagos 3,2.44
612699,A9999,Gordons Lagos 4,A9999_Gordons Lagos 4,1.03
612700,A9999,Gordons Abuja 1,A9999_Gordons Abuja 1,1.22
612701,A9999,Gordons Abuja 4,A9999_Gordons Abuja 4,1.28


In [7]:
df["Joke_identifier"] = df["Joke_identifier"].astype("category").cat.codes
df["Viewers_ID"] = df["Viewers_ID"].astype("category").cat.codes

In [9]:
train = df[:612702]
test = df[612702:]

In [10]:
train.drop(labels=['Response_ID'], axis=1, inplace=True)

/home/olamide/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
x_train, x_test = train_test_split(train, test_size=0.2, random_state=1)

In [12]:
user_id_input = Input(shape=[1], name='user')
item_id_input = Input(shape=[1], name='item')

embedding_size = 15 # 5
user_embedding = Embedding(output_dim=embedding_size, input_dim=x_train["Viewers_ID"].shape[0],
                           input_length=1, name='user_embedding')(user_id_input)
item_embedding = Embedding(output_dim=embedding_size, input_dim=x_train['Joke_identifier'].shape[0],
                           input_length=1, name='item_embedding')(item_id_input)

user_vecs = Reshape([embedding_size])(user_embedding)
item_vecs = Reshape([embedding_size])(item_embedding)


x = Concatenate()([user_vecs, item_vecs])
x = Dropout(0.3)(x)

x = Dense(128, activation='relu', use_bias=True)(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)

y = Dense(1)(x)


model = Model(inputs=[user_id_input, item_id_input], outputs=y)
model.compile(optimizer=optimizers.Adam(lr=0.003), loss='mse')

In [13]:
file_path = 'best_model.hdf5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', save_best_only=True, mode='min')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [14]:
model.fit([x_train["Viewers_ID"], x_train["Joke_identifier"]]
                    , x_train["Rating"]
                    , batch_size=1024, epochs=10
                    , validation_split=0.1
, shuffle=True,callbacks=[checkpoint, earlystop])

Train on 441144 samples, validate on 49017 samples
Epoch 1/10
441144/441144 [==============================] - 81s 184us/step - loss: 5.4714 - val_loss: 4.9160
Epoch 2/10
441144/441144 [==============================] - 87s 197us/step - loss: 4.6405 - val_loss: 4.7558
Epoch 3/10
441144/441144 [==============================] - 87s 198us/step - loss: 4.3697 - val_loss: 4.7131
Epoch 4/10
441144/441144 [==============================] - 87s 198us/step - loss: 4.1625 - val_loss: 4.7007
Epoch 5/10
441144/441144 [==============================] - 86s 194us/step - loss: 4.0300 - val_loss: 4.7135
Epoch 6/10
441144/441144 [==============================] - 76s 172us/step - loss: 3.9273 - val_loss: 4.7619
Epoch 7/10
441144/441144 [==============================] - 75s 170us/step - loss: 3.8428 - val_loss: 4.8162


In [15]:
model = load_model(file_path)

In [16]:
score = model.evaluate([x_test["Viewers_ID"], x_test["Joke_identifier"]], x_test['Rating'],verbose=1)
score

122541/122541 [==============================] - 4s 30us/step


4.68776395141681

In [ ]:
preds = model.predict([test["Viewers_ID"], test["Joke_identifier"]]).astype(float)

In [ ]:
preds = np.around(preds, 2)

In [ ]:
sub["Rating"] = preds
sub.to_csv("submission.csv", index=False)